### LCEL Deepdive

In [ ]:
!pip install -qU langchain-groq langchain_community langchain_huggingface faiss-cpu

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq
#from dotenv import load_dotenv
#load_dotenv()
from google.colab import userdata
import os

os.environ["GROQ_API_KEY"] = userdata.get('GROQ_API_KEY')

In [ ]:
prompt = ChatPromptTemplate.from_template("Explain about {topic} in detail")
#model = ChatOpenAI()
# Initialize the Groq model
model = ChatGroq(
    model="llama-3.1-8b-instant",
    temperature=0,
    max_retries=2,
)
output_parser = StrOutputParser()

chain = prompt | model | output_parser

chain.invoke({"topic": "ice cream"})

In [ ]:
print(prompt.invoke({"topic": "ice cream"}))

In [ ]:
from langchain_core.messages.human import HumanMessage

messages = [HumanMessage(content='tell me a short joke about ice cream')]
model.invoke(messages)

### What is this "|" in Python?

In [ ]:
from abc import ABC, abstractmethod

class CRunnable(ABC):
    def __init__(self):
        self.next = None

    @abstractmethod
    def process(self, data):
        """
        This method must be implemented by subclasses to define
        data processing behavior.
        """
        pass

    def invoke(self, data):
        processed_data = self.process(data)
        if self.next is not None:
            return self.next.invoke(processed_data)
        return processed_data

    def __or__(self, other):
        return CRunnableSequence(self, other)

class CRunnableSequence(CRunnable):
    def __init__(self, first, second):
        super().__init__()
        self.first = first
        self.second = second

    def process(self, data):
        return data

    def invoke(self, data):
        first_result = self.first.invoke(data)
        return self.second.invoke(first_result)



In [ ]:
class AddTen(CRunnable):
    def process(self, data):
        print("AddTen: ", data)
        return data + 10

class MultiplyByTwo(CRunnable):
    def process(self, data):
        print("Multiply by 2: ", data)
        return data * 2

class ConvertToString(CRunnable):
    def process(self, data):
        print("Convert to string: ", data)
        return f"Result: {data}"

In [ ]:
a = AddTen()
b = MultiplyByTwo()
c = ConvertToString()

chain = a | b | c

In [ ]:
result = chain.invoke(10)
print(result)

### Runnables from LangChain

In [ ]:
from langchain_core.runnables import RunnablePassthrough, RunnableLambda, RunnableParallel

In [ ]:
chain = RunnablePassthrough() | RunnablePassthrough () | RunnablePassthrough ()
chain.invoke("hello")

In [ ]:
def input_to_upper(input: str):
    output = input.upper()
    return output

In [ ]:
chain = RunnablePassthrough() | RunnableLambda(input_to_upper) | RunnablePassthrough()
chain.invoke("hello")

In [ ]:
chain = RunnableParallel({"x": RunnablePassthrough(), "y": RunnablePassthrough()})

In [ ]:
chain.invoke("hello")

In [ ]:
chain.invoke({"input": "hello", "input2": "goodbye"})

In [ ]:
chain = RunnableParallel({"x": RunnablePassthrough(), "y": lambda z: z["input2"]})

In [ ]:
chain.invoke({"input": "hello", "input2": "goodbye"})

### Nested chains - now it gets more complicated!

In [ ]:
def find_keys_to_uppercase(input: dict):
    output = input.get("input", "not found").upper()
    return output

In [ ]:
chain = RunnableParallel({"x": RunnablePassthrough() | RunnableLambda(find_keys_to_uppercase), "y": lambda z: z["input2"]})

In [ ]:
chain.invoke({"input": "hello", "input2": "goodbye"})

In [ ]:
chain = RunnableParallel({"x": RunnablePassthrough()})

def assign_func(input):
    return 100

def multiply(input):
    return input * 10

In [ ]:
chain.invoke({"input": "hello", "input2": "goodbye"})

In [ ]:
chain = RunnableParallel({"x": RunnablePassthrough()}).assign(extra=RunnableLambda(assign_func))

In [ ]:
result = chain.invoke({"input": "hello", "input2": "goodbye"})
print(result)

### Combine multiple chains (incl. coercion)

In [ ]:
def extractor(input: dict):
    return input.get("extra", "Key not found")

def cupper(upper: str):
    return str(upper).upper()

new_chain = RunnableLambda(extractor) | RunnableLambda(cupper)

In [ ]:
new_chain.invoke({"extra": "test"})

In [ ]:
final_chain = chain | new_chain
final_chain.invoke({"input": "hello", "input2": "goodbye"})

### Real Work example

In [ ]:
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
#from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_huggingface import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

vectorstore = FAISS.from_texts(
    ["Cats love thuna"], embedding=embeddings
)
retriever = vectorstore.as_retriever()
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template=template)

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    RunnableParallel({"context": retriever | format_docs, "question": RunnablePassthrough()})
    | prompt
    | ChatGroq(model="llama-3.1-8b-instant", temperature=0, max_retries=2)
    | StrOutputParser()
)

In [ ]:
rag_chain.invoke("What do cats like to eat?")

In [ ]:
RunnableParallel({"context": retriever | format_docs, "question": RunnablePassthrough()}).invoke("What do cats like to eat?")

In [ ]:
prompt.invoke({"context": "Cats love thuna", "question": "What do cats like to eat?"})

In [ ]:
model.invoke(prompt.invoke({"context": "Cats love thuna", "question": "What do cats like to eat?"}))